In [293]:
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics
import pandas as pd
import json
import os
import codecs
from unidecode import unidecode
import pickle
import string
import decimal
import sys
import pickle
import pyodbc
import sys
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.externals import joblib
from pandas.io.sql import read_sql
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from nltk.stem.snowball import PortugueseStemmer
import re
import math
import numpy as np
import os.path
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn import preprocessing
import codecs

from scipy.stats import pearsonr
from sklearn.model_selection import cross_val_score
from sklearn import feature_selection

# ML
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor

In [5]:
def getCombinations(params):
    comb = [len(v) for v in params.values()]
    s = 1
    for x in comb:
        s = s * x
    
    return s

In [6]:
def json_to_dataframe(path):
    with codecs.open(path, 'r', encoding='utf-8') as reader:
        d = json.load(reader)
        
    return pd.DataFrame.from_dict(d)

In [7]:
def l2normalize(xdf):
    return pd.DataFrame(preprocessing.normalize(xdf, norm='l2', axis=1, copy=True, return_norm=False),
             columns=xdf.columns, index=xdf.index)

In [8]:
def print_result(gold_values, sys_values, name):
    """Docstring."""
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()

    print()
    print('Similarity evaluation\t' + name)
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.2f}\t\t{:18.2f}'.format(pearson, mse))

In [445]:
def featureSelection(x, y):

    def f_regression(X,Y):
        return feature_selection.f_regression(X,Y,center=False) #center=True (the default) would not work ("ValueError: center=True only allowed for dense data") but should presumably work in general

    featureSelector = feature_selection.SelectKBest(score_func=f_regression,k=4)
    featureSelector.fit(x,y)
    selected_index = [zero_based_index for zero_based_index in list(featureSelector.get_support(indices=True))]
    
    return selected_index

In [448]:
setP1 = json_to_dataframe('C:/Users/Asilva/Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/setP1_processed.json')

In [449]:
setP2 = json_to_dataframe('C:/Users/Asilva/Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/setP2_processed.json')
setP3 = json_to_dataframe('C:/Users/Asilva/Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/setP3_processed.json')

In [450]:
setP1 = pd.concat([setP1, setP2], axis=0)

In [451]:
setP2 = setP3.copy()

In [452]:
setP1 = setP1.sort_values('index', ascending=True)
setP1.drop('index', axis=1, inplace=True)
setP2 = setP2.sort_values('index', ascending=True)
setP2.drop('index', axis=1, inplace=True)
setP1['target'] = [float(x) for x in setP1['target']]
setP2['target'] = [float(x) for x in setP2['target']]

In [453]:
setP1.corr()

,antonyms,embeddings_diff,embeddings_max,embeddings_pca,ngram_proportion,target,tfidf,uncommon_proportion,word_overlap
antonyms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
embeddings_diff,NaN,1.000000,0.488740,0.920143,0.413633,0.405456,0.470050,-0.360695,0.520487
embeddings_max,NaN,0.488740,1.000000,0.359233,0.299323,0.287358,0.309807,-0.259425,0.867714
embeddings_pca,NaN,0.920143,0.359233,1.000000,0.356607,0.337837,0.404733,-0.323429,0.384255
ngram_proportion,NaN,0.413633,0.299323,0.356607,1.000000,0.554951,0.765326,-0.215727,0.347661
target,NaN,0.405456,0.287358,0.337837,0.554951,1.000000,0.647781,-0.203088,0.355776
tfidf,NaN,0.470050,0.309807,0.404733,0.765326,0.647781,1.000000,-0.235459,0.365464
uncommon_proportion,NaN,-0.360695,-0.259425,-0.323429,-0.215727,-0.203088,-0.235459,1.000000,-0.239150
word_overlap,NaN,0.520487,0.867714,0.384255,0.347661,0.355776,0.365464,-0.239150,1.000000


In [454]:
setP1 = setP1.drop('uncommon_proportion', 1)
setP1 = setP1.drop('embeddings_pca', 1)
setP1 = setP1.drop('embeddings_max', 1)
setP1 = setP1.drop('antonyms', 1)

setP2 = setP2.drop('uncommon_proportion', 1)
setP2 = setP2.drop('embeddings_pca', 1)
setP2 = setP2.drop('embeddings_max', 1)
setP2 = setP2.drop('antonyms', 1)

In [455]:
x_train = setP1.drop('target', 1)
y_train = setP1['target']
x_test = setP2.drop('target', 1)
y_test = setP2['target']

In [507]:
# featureSelection(x_train, y_train)

In [457]:
# x_train = l2normalize(x_train)
# x_test = l2normalize(x_test)

In [502]:
model_ml = SVR(kernel='poly', gamma=0.5, epsilon=0.1, C=1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Mean Squared Error
-------		------------------
   0.68		              0.41


In [503]:
model_ml = SVR(kernel='poly', C=1, degree=3)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Mean Squared Error
-------		------------------
   0.67		              0.42


In [504]:
model_ml = SVR(kernel='linear', C=1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Mean Squared Error
-------		------------------
   0.68		              0.42


In [505]:
model_ml = RandomForestRegressor(n_estimators=200,n_jobs=-1,bootstrap=True)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'RandomForest')


Similarity evaluation	RandomForest
Pearson		Mean Squared Error
-------		------------------
   0.64		              0.46


In [460]:
model_ml = linear_model.LinearRegression(n_jobs=-1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'LinearRegression')


Similarity evaluation	LinearRegression
Pearson		Mean Squared Error
-------		------------------
   0.68		              0.41


In [461]:
model_ml = linear_model.Ridge()
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'RidgeRegression')


Similarity evaluation	RidgeRegression
Pearson		Mean Squared Error
-------		------------------
   0.67		              0.42


In [462]:
model_ml = linear_model.BayesianRidge(n_iter=1000,normalize=False)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'BayesianRidgeRegression')


Similarity evaluation	BayesianRidgeRegression
Pearson		Mean Squared Error
-------		------------------
   0.68		              0.41


In [463]:
mlp_param_grid = dict(
    max_iter=[1000],
    momentum=[0,0.1,0.3,0.5,0.7],
    learning_rate_init=[0.012,0.013,0.015,0.017,0.11,0.13,0.21],
    hidden_layer_sizes=[(15,),(25,),(35,),(45,),(55,),(60,)],
    solver=['adam'],
    learning_rate=['invscaling','adaptive'],
    activation=['tanh', 'relu'],
    early_stopping=[True, False],
    shuffle=[True],
    batch_size=[500]
)
model_ml = MLPRegressor()
param_search = GridSearchCV(
            estimator=model_ml, 
            param_grid=mlp_param_grid,
            n_jobs=1,
            scoring='neg_mean_squared_error',
            cv = 2,
            refit=True,
            verbose=1
        )
model_ml = param_search
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'NeuralNetworkRegression')

Fitting 2 folds for each of 1680 candidates, totalling 3360 fits


[Parallel(n_jobs=1)]: Done 3360 out of 3360 | elapsed:  3.9min finished



Similarity evaluation	NeuralNetworkRegression
Pearson		Mean Squared Error
-------		------------------
   0.67		              0.42


In [210]:
scores = cross_val_score(
    model_ml,
    x_train,
    y_train,
    cv=10,
    scoring='neg_mean_squared_error'
)
print("MSQ (cross_validation): %0.2f (+/- %0.2f)"
      % (scores.mean(), scores.std() * 2))

MSQ (cross_validation): -0.42 (+/- 0.08)
